In [13]:
!pip install nnunetv2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:0000:01
   ━━━━━━━

In [ ]:
import os

os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"


In [15]:
!nnUNetv2_print_available_pretrained_models


/bin/bash: line 1: nnUNetv2_print_available_pretrained_models: command not found


In [16]:
import nnunetv2

In [17]:
!pip show nnunetv2


Name: nnunetv2
Version: 2.6.2
Summary: nnU-Net is a framework for out-of-the box image segmentation.
Home-page: https://github.com/MIC-DKFZ/nnUNet
Author: Helmholtz Imaging Applied Computer Vision Lab
Author-email: Fabian Isensee <f.isensee@dkfz-heidelberg.de>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i)

In [ ]:
!nnUNetv2_create_dataset_json -d 700 -i imagesTr -l labelsTr -n "KiTS23" --modalities CT --labels kidney tumor


In [ ]:
!python -m nnunetv2.experiment_planning.dataset_conversion.nnUNetv2_create_dataset_json \
    -d 700 \
    -i /kaggle/input/sample-nnunet-data/nnUNet_raw/Dataset700_KiTS23/imagesTr \
    -l /kaggle/input/sample-nnunet-data/nnUNet_raw/Dataset700_KiTS23/labelsTr \
    -n "KiTS23" \
    --modalities CT \
    --labels kidney tumor


In [ ]:
import os

os.environ["nnUNet_raw"] = "/kaggle/input/nnunet-sample-data/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"

# Confirm
for k, v in os.environ.items():
    if k.startswith("nnUNet"):
        print(k, "=", v)


In [ ]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


In [ ]:
import os
import shutil
from pathlib import Path

imagesTr_dir = "/kaggle/input/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23/imagesTr"
labelsTr_dir = "/kaggle/input/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23/labelsTr"

# Flatten imagesTr
for i, folder in enumerate(os.listdir(imagesTr_dir)):
    folder_path = os.path.join(imagesTr_dir, folder)
    if os.path.isdir(folder_path):
        # Find the .nii or .nii.gz file inside the folder
        nii_files = list(Path(folder_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        # New name: case_00000_0000.nii.gz
        new_name = f"case_{i:05d}_0000.nii.gz"
        new_path = os.path.join(imagesTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        # Remove now-empty folder
        shutil.rmtree(folder_path)

# Flatten labelsTr similarly if needed
for i, file in enumerate(os.listdir(labelsTr_dir)):
    file_path = os.path.join(labelsTr_dir, file)
    if os.path.isdir(file_path):
        nii_files = list(Path(file_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}.nii.gz"
        new_path = os.path.join(labelsTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(file_path)

print("✅ Flattened imagesTr and labelsTr, ready for nnU-Net preprocessing")


In [ ]:
import os
import shutil
from pathlib import Path

# Paths
src_dataset = "/kaggle/input/d/hansanikaumadi/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23"
dst_dataset = "/kaggle/working/"

# Copy entire dataset to working folder
shutil.copytree(src_dataset, dst_dataset)

imagesTr_dir = os.path.join(dst_dataset, "imagesTr")
labelsTr_dir = os.path.join(dst_dataset, "labelsTr")

# Flatten imagesTr
for i, folder in enumerate(os.listdir(imagesTr_dir)):
    folder_path = os.path.join(imagesTr_dir, folder)
    if os.path.isdir(folder_path):
        nii_files = list(Path(folder_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}_0000.nii.gz"
        new_path = os.path.join(imagesTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(folder_path)

# Flatten labelsTr if labels are also in subfolders
for i, file in enumerate(os.listdir(labelsTr_dir)):
    file_path = os.path.join(labelsTr_dir, file)
    if os.path.isdir(file_path):
        nii_files = list(Path(file_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}.nii.gz"
        new_path = os.path.join(labelsTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(file_path)

print("✅ Dataset copied to /kaggle/working, flattened, ready for nnU-Net preprocessing")


In [ ]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


In [ ]:
import json

dataset_json_path = os.path.join(dst_dataset, "dataset.json")

dataset = {
    "labels": {"0": "background", "1": "kidney", "2": "tumor"},
    "channel_names": {"0": "CT"},
    "numTraining": 10,
    "file_ending": ".nii.gz"
}

with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ dataset.json created at:", dataset_json_path)


In [ ]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


In [ ]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


In [ ]:
!cd /kaggle/working/
!rm -rf *

**New Approach**

In [ ]:
import os
import shutil
import json
from pathlib import Path

# ------------------------------
# 1️⃣ Paths
# ------------------------------
src_dataset = "/kaggle/input/d/hansanikaumadi/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23"
dst_dataset = "/kaggle/working/nnUNet_raw/Dataset700_KiTS23"

imagesTr_dir = os.path.join(dst_dataset, "imagesTr")
labelsTr_dir = os.path.join(dst_dataset, "labelsTr")

# ------------------------------
# 2️⃣ Copy dataset to working directory (writable)
# ------------------------------
shutil.copytree(src_dataset, dst_dataset, dirs_exist_ok=True)
print("✅ Dataset copied to working folder:", dst_dataset)

# ------------------------------
# 3️⃣ Flatten imagesTr and labelsTr folders if needed
# ------------------------------
def flatten_folder(folder_path, prefix):
    folder_path = Path(folder_path)
    for i, subfolder in enumerate(folder_path.iterdir()):
        if subfolder.is_dir():
            # find .nii or .nii.gz files inside
            for f in subfolder.glob("*.nii*"):
                new_name = f"{prefix}_{i:03d}_0000.nii.gz" if "images" in folder_path.name else f"{prefix}_{i:03d}.nii.gz"
                shutil.move(str(f), folder_path / new_name)
            # remove now-empty folder
            shutil.rmtree(subfolder)

flatten_folder(imagesTr_dir, "imaging")
flatten_folder(labelsTr_dir, "segmentation")
print("✅ Flattened imagesTr and labelsTr folders")

# ------------------------------
# 4️⃣ Create dataset.json
# ------------------------------
dataset_json_path = os.path.join(dst_dataset, "dataset.json")

training_list = []
for i in range(10):  # adjust if more samples
    training_list.append({
        "image": f"./imagesTr/imaging_{i:03d}_0000.nii.gz",
        "label": f"./labelsTr/segmentation_{i:03d}.nii.gz"
    })

dataset = {
    "labels": {"0": "background", "1": "kidney", "2": "tumor"},
    "channel_names": {"0": "CT"},
    "numTraining": len(training_list),
    "file_ending": ".nii.gz",
    "training": training_list
}

with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ dataset.json created at:", dataset_json_path)

# # ------------------------------
# # 5️⃣ Run nnU-Net preprocessing
# # ------------------------------
# !nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity --nnunet_raw /kaggle/working/nnUNet_raw


In [ ]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


In [18]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


Fingerprint extraction...
Dataset700_KiTS23
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|███████████████████████████████████████████| 10/10 [00:41<00:00,  4.15s/it]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [2.         0.91432617 0.91432617]. 
Current patch size: (56, 192, 192). 
Current median shape: [142.5        497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [

In [10]:
import os

os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"


In [ ]:
!unzip wefwfgth -d dbeghhyj

In [ ]:
!mv /kaggle/working/nnUNet_raw/Dataset700_KiTS23/labelsTr/segmentation_009.nii.gz /kaggle/working/nnUNet_raw/Dataset700_KiTS23/labelsTr/imaging_009.nii.gz

In [11]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


/bin/bash: line 1: nnUNetv2_plan_and_preprocess: command not found


In [ ]:
import json
import os

# Path to your dataset.json
dataset_json_path = "/kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json"

# 1️⃣ Load the JSON
with open(dataset_json_path, "r") as f:
    dataset = json.load(f)

# 2️⃣ Replace "segmentation_" with "imaging_" in each training label
if "training" in dataset:
    for entry in dataset["training"]:
        if "label" in entry and "segmentation_" in entry["label"]:
            entry["label"] = entry["label"].replace("segmentation_", "imaging_")

# 3️⃣ Save updated JSON
with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ Updated dataset.json saved at:", dataset_json_path)


**Mounting google drive**

In [5]:
!pip install gdown

import os

# Example Google Drive shareable link
drive_url = "https://drive.google.com/uc?id=10lnE57Dd_twtRE-nCVSL1uPxHfv8GCVa"

# Where to save it
zip_path = "/kaggle/working/"

# Download from Drive
!gdown "{drive_url}" -O "{zip_path}"

Downloading...
From (original): https://drive.google.com/uc?id=10lnE57Dd_twtRE-nCVSL1uPxHfv8GCVa
From (redirected): https://drive.google.com/uc?id=10lnE57Dd_twtRE-nCVSL1uPxHfv8GCVa&confirm=t&uuid=054e93c5-4f9c-48cc-8f29-1e43f5fa210a
To: /kaggle/working/nnUNet_raw.zip
100%|██████████████████████████████████████| 1.27G/1.27G [00:41<00:00, 30.7MB/s]


In [8]:
!unzip /kaggle/working/nnUNet_raw.zip -d /kaggle/working/

Archive:  /kaggle/working/nnUNet_raw.zip
   creating: /kaggle/working/nnUNet_raw/
   creating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json  
   creating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_000_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_001_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_002_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_003_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_004_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_005_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/imaging_006_0000.nii.gz  
  inflating: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/imagesTr/

In [ ]:
import shutil

folder_path = "/kaggle/working/nnUNet_raw/nnUNet_raw"

# Remove the folder and everything inside
shutil.rmtree(folder_path)

In [7]:
import os

file_path = "/kaggle/working/view?usp=sharing"

if os.path.exists(file_path):
    os.remove(file_path)
    print("✅ File deleted:", file_path)
else:
    print("❌ File not found:", file_path)

✅ File deleted: /kaggle/working/view?usp=sharing
